In [24]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"



es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)


import json
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)


documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma2:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

query = 'I just disovered the course. Can I still join it?'
rag(query)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 42.20it/s]


"Yes, even if you don't register, you're still eligible to submit the homeworks. \n"

In [25]:
query = 'will I receive an email about the acceptance?'
rag(query)

"You don't need to wait for an email confirmation, you were accepted already. You can start learning. \n"

In [17]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          32G   20G   10G  67% /
tmpfs            64M     0   64M   0% /dev
shm              64M  4.0K   64M   1% /dev/shm
/dev/root        29G   23G  6.2G  79% /vscode
/dev/loop3       32G   20G   10G  67% /workspaces
/dev/sda1       118G   92K  112G   1% /tmp


In [18]:
import os
os.environ['HF_HOME'] = '/workspaces/cache/'